First, we'll import pytorch and check if a GPU is available.

In [272]:
import torch
import random
import tqdm
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from FrEIA.modules import *
from FrEIA.framework import *
from pathlib import Path
from torch.utils.data import Dataset, DataLoader

if torch.cuda.is_available():
    print('GPU available')
else:
    print('CPU only')

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

CPU only


In [312]:
batch_size = 1
seq_length = 41
n_features = 1
lstm_hidden = 100
inn_hidden = 128

Next, define the path to the data we're using

In [313]:
if not use_cuda:
    data_path = Path('C:\\Users\\dohert01\\PycharmProjects\\qPAI_cINN_uncertainty_estimation\\datasets')
else:
    data_path = Path('../datasets')

experiment_name = "FlowPhantom_insilico_complicated"

Let's have a look at the data. First of all, borrow some normalisation functions...

In [314]:
def spectrum_normalisation(spectrum):
    """Applies z-score scaling to the initial pressure spectrum"""
    mean = np.mean(spectrum)
    std = np.std(spectrum)
    norm = (spectrum - mean)/std
    return norm

def spectrum_processing(spectrum, allowed_datapoints):
    """Returns a normalised initial pressure spectrum with some of the values zeroed out"""
    num_non_zero_datapoints = random.choice(allowed_datapoints)
    a = np.zeros(len(spectrum))
    a[:num_non_zero_datapoints] = 1
    np.random.shuffle(a)

    incomplete_spectrum = list(np.multiply(a, np.array(spectrum)))
    non_zero_indices = np.nonzero(incomplete_spectrum)
    non_zero_values = list(filter(None,incomplete_spectrum))
    normalised_non_zero = spectrum_normalisation(non_zero_values)

    i = 0
    for index in non_zero_indices[0]:
        incomplete_spectrum[index] = normalised_non_zero[i]
        i+=1

    normalised_incomplete_spectrum = np.array(incomplete_spectrum)

    return normalised_incomplete_spectrum

def batch_spectrum_processing(batch, allowed_datapoints):
    processed = []

    for spectrum in batch:

        processed.append(spectrum_processing(spectrum, allowed_datapoints))
    return torch.tensor(np.array(processed))

Let's load the data from file

In [315]:
training_spectra_file = data_path / experiment_name / "training_spectra.pt"
validation_spectra_file = data_path / experiment_name / "validation_spectra.pt"
test_spectra_file = data_path / experiment_name / "test_spectra.pt"

training_oxygenations_file = data_path / experiment_name / "training_oxygenations.pt"
validation_oxygenations_file = data_path / experiment_name / "validation_oxygenations.pt"
test_oxygenations_file = data_path / experiment_name / "test_oxygenations.pt"

train_spectra_original = torch.load(training_spectra_file)
train_oxygenations_original = torch.load(training_oxygenations_file)
validation_spectra_original = torch.load(validation_spectra_file)
validation_oxygenations_original = torch.load(validation_oxygenations_file)
test_spectra_original = torch.load(test_spectra_file)
test_oxygenations_original = torch.load(test_oxygenations_file)

Now let's look at the dimensions

In [316]:
print(train_spectra_original.size())
print(train_oxygenations_original.size())
print(train_spectra_original[0])

torch.Size([134624, 41])
torch.Size([134624])
tensor([634.9278, 600.2585, 600.2339, 587.4062, 580.4452, 573.9892, 582.9027,
        597.7095, 601.8840, 641.6681, 655.6356, 704.5982, 730.0311, 739.1377,
        762.7631, 768.2003, 789.5642, 808.6349, 811.7870, 835.5294, 866.5328,
        886.8488, 918.7031, 905.1712, 913.7165, 913.7761, 913.4937, 919.7126,
        915.4688, 919.2101, 887.4873, 870.8792, 905.5049, 883.7628, 876.9416,
        888.4904, 881.3424, 888.5063, 892.4427, 879.3855, 869.1013],
       dtype=torch.float64)


In [317]:
# Zeroing out some of the spectrum data (randomly) and normalising
allowed_datapoints = [10]

train_spectra = batch_spectrum_processing(train_spectra_original, allowed_datapoints)
validation_spectra = batch_spectrum_processing(validation_spectra_original, allowed_datapoints)
test_spectra = batch_spectrum_processing(test_spectra_original, allowed_datapoints)

In [318]:
# Reshaping initial pressure spectra to fit LSTM input size
train_spectra = torch.reshape(train_spectra, (len(train_spectra), len(train_spectra[0]), 1))
validation_spectra = torch.reshape(validation_spectra, (len(validation_spectra), len(validation_spectra[0]), 1))
test_spectra = torch.reshape(test_spectra, (len(test_spectra), len(test_spectra[0]), 1))

train_oxygenations = torch.reshape(train_oxygenations_original,(len(train_oxygenations_original),1))
validation_oxygenations = torch.reshape(validation_oxygenations_original,(len(validation_oxygenations_original),1))
test_oxygenations = torch.tensor(np.float32(test_oxygenations_original))
test_oxygenations = torch.reshape(test_oxygenations_original,(len(test_oxygenations_original),1))

In [319]:
class MultiSpectralPressureO2Dataset(Dataset):
    def __init__(self, spectra, oxygenations, transform=None, target_transform=None):
        self.data = spectra
        self.labels = oxygenations
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        data = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            data = self.transform(data)
        if self.target_transform:
            label = self.target_transform(label)
        # TODO - How can I add the input Gaussian to the dataloader? What does the input Gaussian look like? Would it be easier/preferable to input the outputs? Also, probably the Gaussian should be the same every time (?), such that the only information introduced is from the conditioning
        # TODO - Alternativel, can have the Gaussian within the Wrapped Model class and it gets entered on the forward pass.
        return data, label

In [320]:
def switch_seq_feat(tensor):
    # Return a view of the tesor with axes rearranged
    return torch.permute(tensor, (1, 0)).float()
def float_transform(tensor):
    return tensor.float()
training_dataset = MultiSpectralPressureO2Dataset(
    train_spectra, 
    train_oxygenations, 
    transform=switch_seq_feat, 
    target_transform=float_transform
)
training_dataloader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
data, label = next(iter(training_dataloader))

print(data[0].size())
print(label[0].size())
print(data[0].type())
print(label[0].type())
print(data[0])
print(label[0])

torch.Size([1, 41])
torch.Size([1])
torch.FloatTensor
torch.FloatTensor
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  1.3474,  0.0000,  0.0000, -0.0091,
          0.0209,  0.0000,  1.2410,  0.0000,  0.0000,  0.0000,  1.3384,  0.0000,
          0.0000,  0.2498,  0.0000,  0.0000, -0.5825,  0.0000,  0.0000,  0.0000,
          0.0000, -0.9523,  0.0000,  0.0000,  0.0000, -1.2184,  0.0000, -1.4351,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000]])
tensor([0.2914])


In [321]:
# Define the LSTM for the conditioning network
# MASKING lAYER? Input of 2 with 2nd as binary to indicate if zero is real or not
lstm = nn.LSTM(
    input_size=n_features, # Input dimensions
    hidden_size=lstm_hidden, # No. of neurons in gate networks
    batch_first=False
)
class CondNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm =  lstm
        #self.linear = nn.Linear(
        #        in_features=lstm_hidden,
        #        out_features=1
        #)

    def forward(self, x):

        out = self.lstm(x)[0]
        #out = self.linear(out)

        return torch.reshape(out, (batch_size, -1, 1))

# dummy inputs:
x = torch.randn(batch_size, seq_length, n_features)
print(data[0].type())
print(x.size())
print(x.type())

y = lstm(x)
print(f"LSTM output: {y[0].shape}")
cond_network = CondNetwork()
z = cond_network(x)
print(f"Cond output: {z.shape}")

torch.FloatTensor
torch.Size([1, 41, 1])
torch.FloatTensor
LSTM output: torch.Size([1, 41, 100])
Cond output: torch.Size([1, 4100, 1])


Time to define the model... (both LSTM and INN)

In [322]:
# Define the subnet for the invertible blocks (use the AllInOneBlack from FrEIA)
n_blocks = 6  # No. of invertible blocks in INN
def subnet(dims_in, dims_out):
    return nn.Sequential(nn.Linear(dims_in, inn_hidden), nn.LeakyReLU(),
                         nn.Linear(inn_hidden,  inn_hidden), nn.LeakyReLU(),
                         nn.Linear(inn_hidden, dims_out))
tens = torch.randn(1, 1, 62).normal_() # Is randn ok? Should it be seeded (probably for repeatability) or should it not be random at all?
print(tens)
out = subnet(62, 40).forward(tens)
print(out.shape)

tensor([[[-0.9208, -1.2881,  0.4828, -1.1084,  0.9636,  0.7013, -2.8616,
          -0.5026,  0.6625, -0.6185, -0.9619, -2.3585,  1.5150,  0.4384,
           1.1502, -1.5456, -0.3578,  0.5617, -0.0088, -0.6101,  0.1700,
          -0.3819,  0.6710,  0.1099,  1.7232,  0.1913,  0.0604, -2.1134,
          -1.5997, -0.2156, -0.5071, -0.1564,  2.4289, -0.4607, -1.6133,
          -1.1475, -0.1740, -2.3562,  1.4779,  1.2345,  0.1145, -0.2267,
          -0.6371,  0.2498, -0.9373, -0.5317,  0.8264, -0.0949,  0.9595,
          -0.7535,  0.6779, -0.3047,  0.3437,  0.6552, -1.7293,  1.9851,
          -0.0731, -0.2930,  0.0750, -1.4737, -1.1889,  0.4880]]])
torch.Size([1, 1, 40])


The next cell is defined as a SequentialINN...

In [323]:
seq_length_inn = 2
input_dims = (2,)
inn = SequenceINN(*input_dims)

for i in range(n_blocks):
    # TODO - Consider clamping and permute_soft arguments
    inn.append(AllInOneBlock, cond=0, cond_shape=(4100, ), subnet_constructor=subnet)

input = torch.randn(batch_size, seq_length_inn, n_features).normal_()
cond = cond_network(torch.randn(batch_size, seq_length, n_features))
z = inn(input, c=[cond])

RuntimeError: mat1 and mat2 shapes cannot be multiplied (4101x1 and 4101x128)

In [ ]:
class WrappedModel(nn.Module):
    def __init__(self, cond_network, inn):
        super().__init__()

        self.cond_network = cond_network
        self.inn = inn

    def forward(self, x):

        x = torch.permute(x, (0, 2, 1))
        cond = self.cond_network(x)
        #cond = torch.permute(cond, (0,2,1))
        norm = torch.randn(batch_size, 41, 1).normal_()
        z = self.inn(norm, [cond])
        zz = sum(torch.sum(o**2, dim=1) for o in z)
        jac = self.inn.jacobian(run_forward=False)

        return zz, jac

    def reverse_sample(self, z, cond):
        return self.inn(z, cond, rev=True)

model = WrappedModel(cond_network, inn)
if use_cuda:
    model.cuda()
    model = nn.DataParallel(model)

# Time to define the training loop...
# Defining optimizer etc
n_epochs = 10
decay_by = 0.01
weight_decay = 1e-5
betas = (0.9, 0.999)
log10_lr = -4.0  # Log learning rate
lr = 10**log10_lr
params_trainable = (list(filter(lambda p: p.requires_grad, model.inn.parameters()))
                  + list(model.cond_network.parameters()))
gamma = decay_by**(1./n_epochs)
optim = torch.optim.Adam(params_trainable, lr=lr, betas=betas, eps=1e-6, weight_decay=weight_decay)
weight_scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=1, gamma=gamma)


# a very basic training loop
for data, label in training_dataloader:
    """
    iterator = tqdm.tqdm(
        iter(training_dataloader),
        total=len(training_dataloader),
        leave=False,
        mininterval=1.,
        ncols=83
    )
    for x in iterator:
    """
    #print(len(x))
    #print(x[0].size())
    #break
    optim.zero_grad()
    # Send data to GPU (https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel)
    # data, label = data.to(device), label.to(device)
    # pass to INN and get transformed variable z and log Jacobian determinant
    zz, jac = model(data)
    # calculate the negative log-likelihood of the model with a standard normal prior
    neg_log_likeli = 0.5 * zz - jac
    loss = torch.mean(neg_log_likeli) / 40  # tot_output_size
    # backpropagate and update the weights
    loss.backward()
    optim.step()

In [ ]:
inn = SequenceINN(41, 1)

for i in range(n_blocks):
    # TODO - Consider clamping and permute_soft arguments
    inn.append(AllInOneBlock, cond=0, cond_shape=(100, 1), subnet_constructor=subnet)
    
class WrappedModel(nn.Module):
    def __init__(self, cond_network, inn):
        super().__init__()

        self.cond_network = cond_network
        self.inn = inn

    def forward(self, x):

        #cond = [x, self.cond_network(x).squeeze()]
        cond = self.cond_network(x)

        z = self.inn(x, cond)
        zz = sum(torch.sum(o**2, dim=1) for o in z)
        jac = self.inn.jacobian(run_forward=False)

        return zz, jac

    def reverse_sample(self, z, cond):
        return self.inn(z, cond, rev=True)

model = WrappedModel(lstm, inn)
if use_cuda:
    model.cuda()
    model = nn.DataParallel(model)

Time to define the training loop...

In [ ]:
# Defining optimizer etc
n_epochs = 10
decay_by = 0.01
weight_decay = 1e-5
betas = (0.9, 0.999)
log10_lr = -4.0                     # Log learning rate
lr = 10**log10_lr
lr_feature_net = lr                 # lr of the cond. network
params_trainable = list(filter(lambda p: p.requires_grad, model.parameters()))
gamma = decay_by**(1./n_epochs)
optim = torch.optim.Adam(params_trainable, lr=lr, betas=betas, eps=1e-6, weight_decay=weight_decay)
weight_scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=1, gamma=gamma)


In [ ]:
# a very basic training loop
#for data, label in training_dataloader:
iterator = tqdm.tqdm(enumerate(iter(training_dataloader)),
                     total=len(training_dataloader),
                     leave=False,
                     mininterval=1.,
                     ncols=83)
for i_batch, x in iterator:
    #print(len(x))
    #print(x[0].size())
    #break
    optim.zero_grad()
    # Send data to GPU (https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel)
    #data, label = data.to(device), label.to(device)
    # pass to INN and get transformed variable z and log Jacobian determinant
    zz, jac = model.forward(x[0])
    # calculate the negative log-likelihood of the model with a standard normal prior
    neg_log_likeli = 0.5 * zz - jac
    loss = torch.mean(neg_log_likeli) / 41#tot_output_size
    # backpropagate and update the weights
    loss.backward()
    optim.step()